In [ ]:
import couchdb
import spacy
nlp = spacy.load('es_core_news_md')
from spacy.lang.es.stop_words import STOP_WORDS
from spacy.matcher import Matcher
from spacy.tokens import Doc, Span
from spacy.matcher import PhraseMatcher
import string
from spacy.lang.es import Spanish
from spacy.language import Language
import re
import unidecode
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline
import seaborn as sns
import pandas as pd
import numpy as np
from wordcloud import WordCloud
from time import time
import subprocess
import gc
for limpio in range(3):
    gc.collect(generation=limpio)
import gensim
from gensim import corpora
from gensim import models
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim
import unicodedata
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
for limpio in range(3):
        gc.collect(generation=limpio)
import sys

In [ ]:
couch = couchdb.Server('')#colocar la dirección de couchDB en el equipo
dbname = "testimonios"
if dbname in couch:
    db = couch[dbname]
else:
    db = couch.create(dbname)

In [ ]:
otros = ['RJN', 'FAFZ']
sujeto =['CJB', 'RAR', 'PAR', 'RCL', 'JVR', 'CLWP']
sobreviviente = ['CJB', 'RAR', 'PAR']
desaparecido = ['RCL', 'JVR', 'CLWP']
etapa = ['infancia', 'adolescencia', 'toma_de_decision', 'vida_militar', 'hecho_victimizante',   
         'despues_hecho_victimizante', 'familia',]
antes = ['infancia', 'adolescencia', 'toma_de_decision']
durante = ['vida_militar','hecho_victimizante']
despues = ['despues_hecho_victimizante']
familia = ['familia']
lista_color = ['#0000FF', '#008000', '#FF0000', '#00BFBF', '#BF00BF', '#BFBF00']

## Análisis descriptivo

In [ ]:
tiempo = 0
largo = 0
sujetos = []
etapa_rel = []
tiempos = []
extension = []
for i in sujeto:
    for j in etapa:
        for id in db:
            doc = db.get(id)
            if doc['nombre'] == i and doc['etapa'] == j:
                tiempo+= int(round(float(doc['duracion']), 0))
                largo+= int(len(doc['relato']))
        sujetos.append(i)
        etapa_rel.append(j)
        tiempos.append(tiempo)
        extension.append(largo)
        tiempo = 0
        largo = 0

In [ ]:
descriptivos = pd.DataFrame(zip(sujetos, etapa_rel, tiempos, extension), 
                            columns=['sujeto', 'etapa', 'tiempo_audio', 'ext_texto'])
descriptivos.to_excel('descriptivos.xlsx')

In [ ]:
#descriptivos = pd.read_excel('descriptivos.xlsx')
df2 = descriptivos[['sujeto', 'tiempo_audio', 'ext_texto']]
resumen = df2.groupby(['sujeto'], as_index=False).sum()
fig, ax = plt.subplots(1, 2, sharex = True, figsize=(15,10))
fig.suptitle('Distribución de datos por sujeto', fontsize = 20)
ax[0].set_title('Proporción de audios (tiempo)', fontsize = 16)
ax[0].pie(resumen.tiempo_audio, labels=resumen.sujeto, autopct='%1.2f%%', shadow = True,
          colors=lista_color, counterclock = False, explode = (0, 0, 0, 0, 0.1, 0), textprops={"fontsize":14})
#ax[0].axis("equal")
ax[1].set_title('Proporción de textos (palabras)', fontsize = 16)
ax[1].pie(resumen.ext_texto, labels=resumen.sujeto, autopct='%1.2f%%', shadow = True,
          colors=lista_color, counterclock = False, explode = (0, 0, 0, 0, 0.1, 0), textprops={"fontsize":14})
#ax[1].axis("equal")
fig.tight_layout()
plt.show()

In [ ]:
posicion = 0
fig, ax = plt.subplots(2, 3, sharex = True, figsize=(15,10))
fig.suptitle('Tiempo de grabaciones por etapa', fontsize = 16)
for a in range(0, 2):
    for b in range(0, 3):
        df_filtrado = descriptivos[descriptivos['sujeto']==sujeto[posicion]]
        ax[a,b].set_title(sujeto[posicion], fontsize = 14, color = lista_color[posicion])
        ax[a,b].bar(df_filtrado.etapa, df_filtrado.tiempo_audio, color = lista_color[posicion])
        ax[a,b].tick_params(axis="x", labelrotation=75)
        textotal = str(round(df_filtrado['tiempo_audio'].sum()/3600,2)) + ' \nhoras'
        ax[a,b].text(1, max(df_filtrado['tiempo_audio']-(max(df_filtrado['tiempo_audio']*0.1))),
                              textotal, fontsize = 14)
        if a == 0 and b == 0:
            ax[0,0].set_ylabel('Sobrevivientes \n tiempo (segundos)')
        elif a == 1 and b == 0:
            ax[1,0].set_ylabel('Desaparecidos \n tiempo (segundos)')
        else:
            ax[a,b].set_ylabel('tiempo (segundos)')
        posicion += 1
fig.tight_layout()
plt.show()

In [ ]:
posicion = 0
fig, ax = plt.subplots(2, 3, sharex = True, figsize=(15,10))
fig.suptitle('Cantidad de palabras por etapa', fontsize = 16)
for a in range(0, 2):
    for b in range(0, 3):
        df_filtrado = descriptivos[descriptivos['sujeto']==sujeto[posicion]]
        ax[a,b].set_title(sujeto[posicion], fontsize = 14, color = lista_color[posicion])
        ax[a,b].bar(df_filtrado.etapa, df_filtrado.ext_texto, color = lista_color[posicion])
        ax[a,b].tick_params(axis="x", labelrotation=75)
        textotal = str(df_filtrado['ext_texto'].sum()) + ' \npalabras'
        ax[a,b].text(1, max(df_filtrado['ext_texto']-(max(df_filtrado['ext_texto']*0.1))),
                              textotal, fontsize = 14)
        if a == 0 and b == 0:
            ax[0,0].set_ylabel('Sobrevivientes \n Cantidad')
        elif a == 1 and b == 0:
            ax[1,0].set_ylabel('Desaparecidos \n Cantidad')
        else:
            ax[a,b].set_ylabel('Cantidad')
        posicion += 1
fig.tight_layout()
plt.show()

### Limpieza y tokenización de textos

In [ ]:
def limpieza(documento):
    texto = unidecode.unidecode(documento)
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8', 'ignore') 
    texto = texto.replace("\t", " ")
    texto = re.sub("\d+", ' ', texto) #quitar decimales
    regex = '[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]'
    texto = re.sub(regex, "", texto)
    documento = nlp(texto)
    tokens_limpios = [token.lemma_.lower() for token in documento if token.pos_ != "PROPN"]
    tokens_limpios = [token for token in tokens_limpios if not nlp(token)[0].is_punct | nlp(token)[0].is_stop]
    tokens_limpios = [token for token in tokens_limpios if token != " "]
    tokens_limpios = [token for token in tokens_limpios if len(token) > 3]
    return tokens_limpios

In [ ]:
for id in db:
    doc = db.get(id)
    if doc['nombre'] in otros:
        %time doc['tokens'] = limpieza(doc['relato'])
    db.save(doc)
    for limpio in range(3):
        gc.collect(generation=limpio)
subprocess.run('shutdown -h 2', shell=True)

### Cluster

In [ ]:
def extracciontokens(lista_sujetos, etapa):
    """
    función que permite unificar las listas de tokens de las distintas etápas transcritas en el marco de la investigación.
    Para que el proceso funcione, se requiere que esté conectado a couchDB.
    La función requiere dos valores:
    
    1. lista_sujetos = lista de las siglas de nombres de los tokens a extraer.
    2. etapa = corresponde a la lista de las etápas que se van a analizar. 
    
    El retorno de la función es una lista donde estarían todos los tokens del sujeto con base en la etapa 
    elegida (antes, durante, después o familia). 
    """
    palabras = []
    lista_tokens = []
    for id in db:
        doc = db.get(id)
        if doc['nombre'] in lista_sujetos:
            if doc['etapa'] in etapa:
                lista = doc['tokens']
                lista = [i for i in lista if i != " "]
                palabras.append(lista)
    return palabras

In [ ]:
bolsa = extracciontokens(sujeto, etapa)

In [ ]:
def armado_corpus(lista_tokens, tipo_modelo = 'token'):
    """
    La función crea los modelos por token (por defecto), bigramas o trigramas.
    Los datos que requiere son:
    lista_tokens = listado de palabras preprocesadas para realizar el modelo.
    tipo_modelo = se debe escribir 'token' en caso de modelar por unigrama, 'bigram' para modelas bigramas y 'trigram'
    para modelar trigramas. Por defecto modela por unigramas.
    """
    if tipo_modelo == 'bigram':
        bigram = gensim.models.Phrases(bolsa, min_count=5, threshold= 100)
        bigram_mod = gensim.models.phrases.Phraser(bigram)
        bolsa_mod = [bigram_mod[doc] for doc in lista_tokens]
        diccionario = corpora.Dictionary(bolsa_mod)
        diccionario.filter_extremes(no_below = 2, no_above = 0.8)
        corpus = [diccionario.doc2bow(i) for i in lista_tokens]
    elif tipo_modelo == 'trigram':
        bigram = gensim.models.Phrases(bolsa, min_count=5, threshold= 100)
        trigram = gensim.models.Phrases(bigram[bolsa], threshold=100)
        trigram_mod = gensim.models.phrases.Phraser(trigram)
        bolsa_mod = [trigram_mod[doc] for doc in lista_tokens]
        diccionario = corpora.Dictionary(bolsa_mod)
        diccionario.filter_extremes(no_below = 2, no_above = 0.8)
        corpus = [diccionario.doc2bow(i) for i in lista_tokens]
    elif tipo_modelo == 'token':
        bolsa_mod = lista_tokens
        diccionario = corpora.Dictionary(lista_tokens)
        diccionario.filter_extremes(no_below = 2, no_above = 0.8)
        corpus = [diccionario.doc2bow(i) for i in lista_tokens]
    return diccionario, corpus, bolsa_mod

In [ ]:
diccionario, corpus, bolsa = armado_corpus(bolsa, 'token')

# Entrenamiento de modelo

## 16 documentos (20%) por pase con 20 pases.

In [ ]:
topicos = []
coherencia = []
modelo = []
for topico in range(2, 10):
    
    modelo_lda = models.LdaModel(corpus=corpus, id2word=diccionario, num_topics= topico, random_state=100, 
                                 update_every=1, chunksize=16,passes=20, alpha='auto')
    modelo.append(modelo_lda)
    coherencia_modelo_lda = CoherenceModel(model = modelo_lda, texts= bolsa, dictionary=diccionario,
                                      coherence='c_v')
    coherencia.append(coherencia_modelo_lda.get_coherence())
    topicos.append(topico)

fig = plt.figure(figsize = (10,5))
plt.plot(topicos, coherencia)
plt.xticks(range(2, max(topicos)+1))
plt.xlabel('cantidad de tópicos')
plt.ylabel('coherencia')
plt.title('Estimación de la cantidad de tópicos')
plt.grid()
plt.show()

In [ ]:
estimacion1 = pd.DataFrame(zip(['Modelo 1', 'Modelo 2', 'Modelo 3', 'Modelo 4', 'Modelo 5', 'Modelo 6'], topicos, coherencia), columns = ['Modelo entrenado', 'Cant. tópicos', 'Coherencia'])
estimacion1.to_excel('tabla_estimacion_1.xlsx')

### Visualización modelo de 7 tópicos

In [ ]:
pyLDAvis.enable_notebook()
vis3 = pyLDAvis.gensim.prepare(modelo[5], corpus, diccionario) 
vis3

### Visualización modelo de 6 tópicos

In [ ]:
pyLDAvis.enable_notebook()
vis2 = pyLDAvis.gensim.prepare(modelo[4], corpus, diccionario) 
vis2

### Visualización modelo de 5 tópicos

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(modelo[3], corpus, diccionario) 
vis

## 20 documentos (25%) por pase con 30 pases.

In [ ]:
topicosb = []
coherenciab = []
modelob = []
for topico in range(2, 10):
    
    modelo_lda = models.LdaModel(corpus=corpus, id2word=diccionario, num_topics= topico, random_state=100, 
                                 update_every=1, chunksize=20,passes=30, alpha='auto')
    modelob.append(modelo_lda)
    coherencia_modelo_lda = CoherenceModel(model = modelo_lda, texts= bolsa, dictionary=diccionario,
                                      coherence='c_v')
    coherenciab.append(coherencia_modelo_lda.get_coherence())
    topicosb.append(topico)
fig = plt.figure(figsize = (10,5))
#figure(figsize=(40/2.54, 10/2.54))
plt.plot(topicosb, coherenciab)
plt.xticks(range(2, max(topicosb)+1))
plt.xlabel('cantidad de tópicos')
plt.ylabel('coherencia')
plt.title('Estimación de la cantidad de tópicos')
plt.grid()
plt.show()

In [ ]:
estimacion2 = pd.DataFrame(zip(['Modelo 1', 'Modelo 2', 'Modelo 3', 'Modelo 4', 'Modelo 5', 'Modelo 6'], topicosb, coherenciab), columns = ['Modelo entrenado', 'Cant. tópicos', 'Coherencia'])
estimacion2.to_excel('tabla_estimacion_2.xlsx')

### Visualización modelo de 7 tópicos

In [ ]:
pyLDAvis.enable_notebook()
vis6 = pyLDAvis.gensim.prepare(modelob[5], corpus, diccionario) 
vis6

### Visualización modelo de 6 tópicos

In [ ]:
pyLDAvis.enable_notebook()
vis5 = pyLDAvis.gensim.prepare(modelob[4], corpus, diccionario) 
vis5

### Visualización modelo de 5 tópicos

In [ ]:
pyLDAvis.enable_notebook()
vis4 = pyLDAvis.gensim.prepare(modelob[3], corpus, diccionario) 
vis4

In [ ]:
topicos = []
for indice, topic in modelo[3].print_topics(num_words=20):
    topico = topic.split(' + ')
    topicos.append(topico)
basetopicos = pd.DataFrame(topicos)
basetopicos = basetopicos.transpose()
basetopicos.columns = ['Topico_1', 'Topico_2', 'Topico_3', 'Topico_4', 'Topico_5']
columnas = list(basetopicos.columns)
for i in columnas:
    listatopic = basetopicos[i].str.split('*', expand=True)
    listatopic.columns = ['%_' + i.lower(), i.lower()]
    basetopicos = pd.concat([basetopicos, listatopic], axis = 1)
basetopicos = basetopicos.drop(columnas, axis=1)
basetopicos.to_excel('base_topicos_modelo_A.xlsx')
basetopicos.head(10)

In [ ]:
topicos = []
for indice, topic in modelob[3].print_topics(num_words=20):
    topico = topic.split(' + ')
    topicos.append(topico)
basetopicos = pd.DataFrame(topicos)
basetopicos = basetopicos.transpose()
basetopicos.columns = ['Topico_1', 'Topico_2', 'Topico_3', 'Topico_4', 'Topico_5']
columnas = list(basetopicos.columns)
for i in columnas:
    listatopic = basetopicos[i].str.split('*', expand=True)
    listatopic.columns = ['%_' + i.lower(), i.lower()]
    basetopicos = pd.concat([basetopicos, listatopic], axis = 1)
basetopicos = basetopicos.drop(columnas, axis=1)
basetopicos.to_excel('base_topicos_modelo_B.xlsx')
basetopicos.head(10)

# Tópicos presentes en los textos

## Sobrevivientes

In [ ]:
lda=modelob[3]

In [ ]:
fig, ax = plt.subplots(3, figsize=(10,8))
fig.suptitle('Tópicos más importantes en los relatos antes del hecho victimizante')
colores = lista_color[0:3]
for s in range(0,3):
    dato_sujeto = sobreviviente[s]
    relato1 = extracciontokens(dato_sujeto, antes)
    diccionario1, corpus1, bolsa1 = armado_corpus(relato1)
    dist_indices = []
    dist_topicos = []
    dist_contrib = []
    lista_topicos = ['Hecho vict.', 'Infancia', 'Después hech.', 'Proyecto vida', 'Vida mil.']
    for topico in lda[corpus1]:
        for i in range(0, len(topico)):
            dist_indices.append(topico[i][0])
            dist_topicos.append(lista_topicos[topico[i][0]])
            dist_contrib.append(topico[i][1])
    distribucion_topicos = pd.DataFrame(list(zip(dist_indices, dist_topicos, dist_contrib)), columns=['# de Topico', 'Topico', 'Contribucion'])
    distribucion_topicos = distribucion_topicos.groupby(by='Topico', as_index=False).mean()
    distribucion_topicos.sort_values('Contribucion', ascending=False, inplace=True)
    distribucion_topicos.to_excel('distribucion_topicos_antes_'+ dato_sujeto + '.xlsx')
    color = colores[s]
    ax[s].bar(distribucion_topicos.Topico, distribucion_topicos.Contribucion, color = color)
    ax[s].set_title(str(dato_sujeto))
    for a in ax.flat:
        a.set(xlabel='Tópicos', ylabel='Contribución(promedio)')
    fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(3, figsize=(10,8))
fig.suptitle('Tópicos más importantes en los relatos durante del hecho victimizante')
colores = lista_color[0:3]
lista_topicos = ['Hecho vict.', 'Infancia', 'Después hech.', 'Proyecto vida', 'Vida mil.']
for s in range(0,3):
    dato_sujeto = sobreviviente[s]
    relato2 = extracciontokens(dato_sujeto, durante)
    diccionario2, corpus2, bolsa2 = armado_corpus(relato2)
    dist_indices = []
    dist_topicos = []
    dist_contrib = []
    for topico in lda[corpus2]:
        for i in range(0, len(topico)):
            dist_indices.append(topico[i][0])
            dist_topicos.append(lista_topicos[topico[i][0]])
            dist_contrib.append(topico[i][1])
    distribucion_topicos = pd.DataFrame(list(zip(dist_indices, dist_topicos, dist_contrib)), columns=['# de Topico', 'Topico', 'Contribucion'])
    distribucion_topicos = distribucion_topicos.groupby(by='Topico', as_index=False).mean()
    distribucion_topicos.sort_values('Contribucion', ascending=False, inplace=True)
    distribucion_topicos.to_excel('distribucion_topicos_durante_'+ dato_sujeto + '.xlsx')
    color = colores[s]
    ax[s].bar(distribucion_topicos.Topico, distribucion_topicos.Contribucion, color = color)
    ax[s].set_title(str(dato_sujeto))
    for a in ax.flat:
        a.set(xlabel='Tópicos', ylabel='Contribución(promedio)')
    fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(3, figsize=(10,8))
fig.suptitle('Tópicos más importantes en los relatos después del hecho victimizante')
colores = lista_color[0:3]
for s in range(0,3):
    dato_sujeto = sobreviviente[s]
    relato3 = extracciontokens(dato_sujeto, despues)
    diccionario3, corpus3, bolsa3 = armado_corpus(relato3)
    dist_indices = []
    dist_topicos = []
    dist_contrib = []
    for topico in lda[corpus3]:
        for i in range(0, len(topico)):
            dist_indices.append(topico[i][0])
            dist_topicos.append(lista_topicos[topico[i][0]])
            dist_contrib.append(topico[i][1])
    distribucion_topicos = pd.DataFrame(list(zip(dist_indices, dist_topicos, dist_contrib)), columns=['# de Topico', 'Topico', 'Contribucion'])
    distribucion_topicos = distribucion_topicos.groupby(by='Topico', as_index=False).mean()
    distribucion_topicos.sort_values('Contribucion', ascending=False, inplace=True)
    distribucion_topicos.to_excel('distribucion_topicos_despues_'+ dato_sujeto + '.xlsx')
    color = colores[s]
    ax[s].bar(distribucion_topicos.Topico, distribucion_topicos.Contribucion, color = color)
    ax[s].set_title(str(dato_sujeto))
    for a in ax.flat:
        a.set(xlabel='Tópicos', ylabel='Contribución(promedio)')
    fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(3, figsize=(10,8))
fig.suptitle('Tópicos más importantes en los relatos de familiares de sujetos sobrevivientes')
colores = lista_color[0:3]
for s in range(0,3):
    dato_sujeto = sobreviviente[s]
    relato4 = extracciontokens(dato_sujeto, familia)
    diccionario4, corpus4, bolsa4 = armado_corpus(relato4)
    dist_indices = []
    dist_topicos = []
    dist_contrib = []
    for topico in lda[corpus4]:
        for i in range(0, len(topico)):
            dist_indices.append(topico[i][0])
            dist_topicos.append(lista_topicos[topico[i][0]])
            dist_contrib.append(topico[i][1])
    distribucion_topicos = pd.DataFrame(list(zip(dist_indices, dist_topicos, dist_contrib)), columns=['# de Topico', 'Topico', 'Contribucion'])
    distribucion_topicos = distribucion_topicos.groupby(by='Topico', as_index=False).mean()
    distribucion_topicos.sort_values('Contribucion', ascending=False, inplace=True)
    distribucion_topicos.to_excel('distribucion_topicos_familia_'+ dato_sujeto + '.xlsx')
    color = colores[s]
    ax[s].bar(distribucion_topicos.Topico, distribucion_topicos.Contribucion, color = color)
    ax[s].set_title(str(dato_sujeto))
    for a in ax.flat:
        a.set(xlabel='Tópicos', ylabel='Contribución(promedio)')
    fig.tight_layout()
plt.show()

### Desaparecidos

In [ ]:
fig, ax = plt.subplots(3, figsize=(10,8))
fig.suptitle('Tópicos más importantes en los relatos antes del hecho victimizante')
colores = lista_color[3:6]
for s in range(0,3):
    dato_sujeto = desaparecido[s]
    relato5 = extracciontokens(dato_sujeto, antes)
    diccionario5, corpus5, bolsa5 = armado_corpus(relato5)
    dist_indices = []
    dist_topicos = []
    dist_contrib = []
    for topico in lda[corpus5]:
        for i in range(0, len(topico)):
            dist_indices.append(topico[i][0])
            dist_topicos.append(lista_topicos[topico[i][0]])
            dist_contrib.append(topico[i][1])
    distribucion_topicos = pd.DataFrame(list(zip(dist_indices, dist_topicos, dist_contrib)), columns=['# de Topico', 'Topico', 'Contribucion'])
    distribucion_topicos = distribucion_topicos.groupby(by='Topico', as_index=False).mean()
    distribucion_topicos.sort_values('Contribucion', ascending=False, inplace=True)
    distribucion_topicos.to_excel('distribucion_topicos_antes_'+ dato_sujeto + '.xlsx')
    color = colores[s]
    ax[s].bar(distribucion_topicos.Topico, distribucion_topicos.Contribucion, color = color)
    ax[s].set_title(str(dato_sujeto))
    for a in ax.flat:
        a.set(xlabel='Tópicos', ylabel='Contribución(promedio)')
    fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(3, figsize=(10,8))
fig.suptitle('Tópicos más importantes en los relatos durante del hecho victimizante')
colores = lista_color[3:6]
for s in range(0,3):
    dato_sujeto = desaparecido[s]
    relato6 = extracciontokens(dato_sujeto, durante)
    diccionario6, corpus6, bolsa6 = armado_corpus(relato6)
    dist_indices = []
    dist_topicos = []
    dist_contrib = []
    for topico in lda[corpus6]:
        for i in range(0, len(topico)):
            dist_indices.append(topico[i][0])
            dist_topicos.append(lista_topicos[topico[i][0]])
            dist_contrib.append(topico[i][1])
    distribucion_topicos = pd.DataFrame(list(zip(dist_indices, dist_topicos, dist_contrib)), columns=['# de Topico', 'Topico', 'Contribucion'])
    distribucion_topicos = distribucion_topicos.groupby(by='Topico', as_index=False).mean()
    distribucion_topicos.sort_values('Contribucion', ascending=False, inplace=True)
    distribucion_topicos.to_excel('distribucion_topicos_durante_'+ dato_sujeto + '.xlsx')
    color = colores[s]
    ax[s].bar(distribucion_topicos.Topico, distribucion_topicos.Contribucion, color = color)
    ax[s].set_title(str(dato_sujeto))
    for a in ax.flat:
        a.set(xlabel='Tópicos', ylabel='Contribución(promedio)')
    fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(3, figsize=(10,8))
fig.suptitle('Tópicos más importantes en los relatos despues del hecho victimizante')
colores = lista_color[3:6]
for s in range(0,3):
    dato_sujeto = desaparecido[s]
    relato7 = extracciontokens(dato_sujeto, despues)
    diccionario7, corpus7, bolsa7 = armado_corpus(relato7)
    dist_indices = []
    dist_topicos = []
    dist_contrib = []
    for topico in lda[corpus7]:
        for i in range(0, len(topico)):
            if type(topico) is tuple:
                dist_indices.append(topico[0])
                dist_topicos.append(lista_topicos[topico[0]])
                dist_contrib.append(topico[1])
            else:
                dist_indices.append(topico[i][0])
                dist_topicos.append(lista_topicos[topico[i][0]])
                dist_contrib.append(topico[i][1])
    distribucion_topicos = pd.DataFrame(list(zip(dist_indices, dist_topicos, dist_contrib)), 
                                        columns=['# de Topico', 'Topico', 'Contribucion'])   
    distribucion_topicos = distribucion_topicos.groupby(by='Topico', as_index=False).mean()
    distribucion_topicos.sort_values('Contribucion', ascending=False, inplace=True)
    distribucion_topicos.to_excel('distribucion_topicos_despues_'+ dato_sujeto + '.xlsx')
    color = colores[s]
    ax[s].bar(distribucion_topicos.Topico, distribucion_topicos.Contribucion, color = color)
    ax[s].set_title(str(dato_sujeto))
    for a in ax.flat:
        a.set(xlabel='Tópicos', ylabel='Contribución(promedio)')
    fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(3, figsize=(10,8))
fig.suptitle('Tópicos más importantes en los relatos de familiares')
colores = lista_color[3:6]
for s in range(0,3):
    dato_sujeto = desaparecido[s]
    relato8 = extracciontokens(dato_sujeto, familia)
    diccionario8, corpus8, bolsa8 = armado_corpus(relato8)
    dist_indices = []
    dist_topicos = []
    dist_contrib = []
    for topico in lda[corpus8]:
        for i in range(0, len(topico)):
            if type(topico) is tuple:
                dist_indices.append(topico[0])
                dist_topicos.append(lista_topicos[topico[0]])
                dist_contrib.append(topico[1])
            else:
                dist_indices.append(topico[i][0])
                dist_topicos.append(lista_topicos[topico[i][0]])
                dist_contrib.append(topico[i][1])
    distribucion_topicos = pd.DataFrame(list(zip(dist_indices, dist_topicos, dist_contrib)), columns=['# de Topico', 'Topico', 'Contribucion'])
    distribucion_topicos = distribucion_topicos.groupby(by='Topico', as_index=False).mean()
    distribucion_topicos.sort_values('Contribucion', ascending=False, inplace=True)
    distribucion_topicos.to_excel('distribucion_topicos_familia_'+ dato_sujeto + '.xlsx')
    color = colores[s]
    ax[s].bar(distribucion_topicos.Topico, distribucion_topicos.Contribucion, color = color)
    ax[s].set_title(str(dato_sujeto))
    for a in ax.flat:
        a.set(xlabel='Tópicos', ylabel='Contribución(promedio)')
    fig.tight_layout()
plt.show()